In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re

poi_df = pd.read_csv('/content/gdrive/MyDrive/cleaned_Job-rec/poi_cleaned.csv')
job_views_df = pd.read_csv('/content/gdrive/MyDrive/cleaned_Job-rec/job_views_cleaned.csv')
experience_df = pd.read_csv('/content/gdrive/MyDrive/Job-rec/Experience.csv')
final_df = pd.read_csv('/content/gdrive/MyDrive/Job-rec/Combined_Jobs_Final.csv')

experience_df['Can.Contact.Employer'] = experience_df['Can.Contact.Employer'].fillna(False)
# create a dictionary of state names and codes
state_dict = {
    'California': 'CA',
    'New York': 'NY',
    'Texas': 'TX',
    "Phoenix": ("Arizona", "AZ"),
    "Philadelphia": ("Pennsylvania", "PA"),
    "San Antonio": ("Texas", "TX"),
    "San Diego": ("California", "CA"),
    "Dallas": ("Texas", "TX"),
    "San Jose": ("California", "CA"),
    "Austin": ("Texas", "TX"),
    "Jacksonville": ("Florida", "FL"),
    "Fort Worth": ("Texas", "TX"),
    "Columbus": ("Ohio", "OH"),
    "San Francisco": ("California", "CA"),
    "Charlotte": ("North Carolina", "NC"),
    "Indianapolis": ("Indiana", "IN"),
    "Seattle": ("Washington", "WA"),
    "Denver": ("Colorado", "CO"),
    "Washington": ("District of Columbia", "DC"),
    "Boston": ("Massachusetts", "MA"),
    "Nashville": ("Tennessee", "TN"),
    "El Paso": ("Texas", "TX"),
    "Detroit": ("Michigan", "MI"),
    "Memphis": ("Tennessee", "TN"),
    "Portland": ("Oregon", "OR"),
    "Oklahoma City": ("Oklahoma", "OK"),
    "Las Vegas": ("Nevada", "NV"),
    "Louisville": ("Kentucky", "KY"),
    "Baltimore": ("Maryland", "MD"),
    "Milwaukee": ("Wisconsin", "WI"),
    "Albuquerque": ("New Mexico", "NM"),
    "Tucson": ("Arizona", "AZ"),
    "Fresno": ("California", "CA"),
    "Sacramento": ("California", "CA"),
    "Mesa": ("Arizona", "AZ"),
    "Atlanta": ("Georgia", "GA"),
    "Kansas City": ("Missouri", "MO"),
    "Colorado Springs": ("Colorado", "CO"),
    "Miami": ("Florida", "FL"),
    "Raleigh": ("North Carolina", "NC"),
    "Omaha": ("Nebraska", "NE"),
    "Long Beach": ("California", "CA"),
    "Virginia Beach": ("Virginia", "VA"),
    "Oakland": ("California", "CA"),
    "Minneapolis": ("Minnesota", "MN"),
    "Tulsa": ("Oklahoma", "OK"),
    "Wichita": ("Kansas", "KS"),
    "New Orleans": ("Louisiana", "LA"),
    "Arlington": ("Texas", "TX"),
}

# replace null values with the corresponding state name and code based on the 'City' column
experience_df['State.Name'].fillna(experience_df['City'].map(lambda x: state_dict.get(x, None)), inplace=True)
experience_df['State.Code'].fillna(experience_df['City'].map(lambda x: state_dict.get(x, None)), inplace=True)
experience_df['City'] = experience_df.groupby('Employer.Name')['City'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
experience_df['Job.Description'] = experience_df.groupby('Position.Name')['Job.Description'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

final_df = final_df.drop("Address", axis=1)
final_df = final_df.drop("Requirements", axis=1)
final_df = final_df.drop("Salary", axis=1)
# Import necessary libraries
from geopy.geocoders import Nominatim

# Create a geolocator object
geolocator = Nominatim(user_agent="my_application")

# Define a function to get city name from latitude and longitude values
def get_city(lat, long):
    location = geolocator.reverse(f"{lat}, {long}")
    address = location.raw['address']
    city = address.get('city', '')
    return city

# Fill the null values in the "City" column with the help of the "Latitude" and "Longitude" columns
final_df['City'] = final_df.apply(lambda x: get_city(x['Latitude'], x['Longitude']) if pd.isnull(x['City']) else x['City'], axis=1)
final_df = final_df.drop(["Latitude", "Longitude"], axis=1)
final_df = final_df.drop(["Listing.Start", "Created.At", "Updated.At", "Listing.End"], axis=1)
final_df = final_df.drop(final_df.tail(90).index)


In [ ]:
poi_df.dropna(subset=['Position.Of.Interest'], inplace=True)


In [ ]:
from scipy.sparse import coo_matrix

user_items = coo_matrix((job_views_df['View.Duration'], (job_views_df['Applicant.ID'], job_views_df['Job.ID'])))

In [ ]:
pip install implicit


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=50)
model.fit(user_items)


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.006926298141479492 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import numpy as np
import os
from implicit.als import AlternatingLeastSquares

from scipy.sparse import coo_matrix
user_items = coo_matrix((job_views_df['View.Duration'], (job_views_df['Applicant.ID'], job_views_df['Job.ID'])))

model = AlternatingLeastSquares(factors=50)
model.fit(user_items)
# Define batch size
batch_size = 1000

# Convert job attributes to a sparse matrix
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Precompute similarities and store in a file
if not os.path.exists('similarities.npz'):
    # Iterate over batches of data
    cosine_similarities = None
    for i in range(0, len(final_df), batch_size):
        job_attributes = final_df.iloc[i:i+batch_size][['Title', 'Position', 'Company', 'City', 'State.Name', 'Industry', 'Job.Description']].fillna('')
        job_matrix = tfidf.fit_transform(job_attributes.apply(lambda x: ' '.join(x), axis=1))
        job_matrix_sparse = sparse.csr_matrix(job_matrix)
        batch_cosine_similarities = cosine_similarity(job_matrix_sparse)
        if cosine_similarities is None:
            cosine_similarities = batch_cosine_similarities
        else:
            cosine_similarities = np.concatenate((cosine_similarities, batch_cosine_similarities), axis=0)
    np.savez_compressed('similarities.npz', cosine_similarities=cosine_similarities)
else:
    cosine_similarities = np.load('similarities.npz')['cosine_similarities']

def get_recommendations_cb(position_of_interest, cosine_similarities, job_attributes, final_df, top_n=10):
    idx = final_df[final_df['Position'] == position_of_interest].index[0]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    job_indices = [i[0] for i in sim_scores]
    return final_df.iloc[job_indices][['Title', 'Position', 'Company', 'City', 'State.Name', 'Industry', 'Job.Description']]



/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0011844635009765625 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
def get_recommendations_hybrid(user_id, model, job_attributes, final_df, top_n=10):
    job_scores = model.user_factors[user_id].dot(job_matrix.T)
    job_scores = np.squeeze(np.asarray(job_scores))
    job_scores.all()
    top_job_indices_cf = np.argpartition(-job_scores, top_n)[:top_n]
    top_job_indices_cf.all()
    top_jobs_cf = final_df.iloc[top_job_indices_cf][['Title', 'Position', 'Company', 'City', 'State.Name', 'Industry', 'Job.Description']]
    top_jobs_cb = get_recommendations_cb(poi_df[poi_df['Applicant.ID'] == user_id]['Position.Of.Interest'].iloc[0], cosine_similarities, job_attributes, final_df, top_n=top_n)
    hybrid_scores = (0.5 * job_scores[top_job_indices_cf]) + (0.5 * cosine_similarities[top_job_indices_cf, :][:, top_jobs_cb.index].sum(axis=1))
    top_job_indices_hybrid = np.argsort(hybrid_scores)[::-1][:top_n]
    top_jobs_hybrid = final_df.iloc[top_job_indices_hybrid][['Title', 'Position', 'Company', 'City', 'State.Name', 'Industry', 'Job.Description']]
    return top_jobs_hybrid

In [ ]:
def get_recommendations_hybrid(user_id, model, job_attributes, final_df, top_n=10):
    job_scores = model.user_factors[user_id].dot(job_matrix.T)
    job_scores = np.squeeze(np.asarray(job_scores))
    
    # Split the job_scores array into smaller arrays of size top_n
    job_score_chunks = [job_scores[i:i+top_n] for i in range(0, len(job_scores), top_n)]
    
    # Sort each of the smaller arrays separately to get the top_n elements
    top_job_indices_cf = np.concatenate([np.argsort(chunk)[::-1][:top_n] for chunk in job_score_chunks])
    
    top_jobs_cf = final_df.iloc[top_job_indices_cf][['Title', 'Position', 'Company', 'City', 'State.Name', 'Industry', 'Job.Description']]
    top_jobs_cb = get_recommendations_cb(poi_df[poi_df['Applicant.ID'] == user_id]['Position.Of.Interest'].iloc[0], cosine_similarities, job_attributes, final_df, top_n=top_n)

    return top_jobs_cf, top_jobs_cb


In [ ]:
get_recommendations_hybrid(10003)


ValueError: ignored

In [ ]:
user_id = 9970
get_recommendations_hybrid(user_id, model, job_attributes, final_df, top_n=10)

ValueError: ignored

In [ ]:
def get_recommendations_hybrid(applicant_id, top_n=10):
    # Get the position of interest for the applicant
    poi = poi_df[poi_df['Applicant.ID'] == applicant_id]['Position.Of.Interest'].iloc[0]
    
    # Get the top 5 job recommendations using collaborative filtering
    user_items_sparse = user_items.T.tocsr()
    user_index = job_views_df[job_views_df['Applicant.ID'] == applicant_id]['Applicant.ID'].iloc[0]
    recommendations = model.recommend(user_index, user_items_sparse, N=5, filter_already_liked_items=True)
    job_indices_cf = [recommendation[0] for recommendation in recommendations]

    # Get the top 5 job recommendations using content-based filtering
    job_attributes = final_df[['Title', 'Position', 'Company', 'City', 'State.Name', 'Industry', 'Job.Description']].fillna('')
    job_matrix = tfidf.transform(job_attributes.apply(lambda x: ' '.join(x), axis=1))
    job_matrix_sparse = sparse.csr_matrix(job_matrix)
    similarities = cosine_similarity(job_matrix_sparse)
    job_indices_cb = get_recommendations_cb(poi, similarities, job_attributes, final_df, top_n=5).index

    # Get the hybrid job recommendations
    job_indices = set(job_indices_cf).union(set(job_indices_cb))
    job_indices = list(job_indices)[:top_n]
    return final_df.iloc[job_indices][['Title', 'Position', 'Company', 'City', 'State.Name', 'Industry', 'Job.Description']]
